In [ ]:
import os
import torch
from huggingface_hub import hf_hub_download
from pipeline_flux import FluxPipeline
from transformer_flux import FluxTransformer2DModel
from patch_conv import convert_model

In [ ]:
bfl_repo="black-forest-labs/FLUX.1-dev"
device = torch.device('cuda')
dtype = torch.bfloat16
transformer = FluxTransformer2DModel.from_pretrained(bfl_repo, subfolder="transformer", torch_dtype=dtype)
pipe = FluxPipeline.from_pretrained(bfl_repo, transformer=transformer, torch_dtype=dtype)
pipe.scheduler.config.use_dynamic_shifting = False
pipe.scheduler.config.time_shift = 10
pipe.enable_model_cpu_offload()

In [ ]:
if not os.path.exists('ckpt/urae_4k_adapter_lora_conversion_dev.safetensors'):
    hf_hub_download(repo_id="Huage001/URAE", filename='urae_4k_adapter_lora_conversion_dev.safetensors', local_dir='ckpt', local_dir_use_symlinks=False)
pipe.load_lora_weights("ckpt/urae_4k_adapter_lora_conversion_dev.safetensors")

* Use patch-wise convolution for VAE to avoid OOM error when decoding
* If still OOM, try replacing the following line with `pipe.vae.enable_tiling()`

In [ ]:
pipe.vae = convert_model(pipe.vae, splits=4)

In [ ]:
prompt = "A serene woman in a flowing azure dress, gracefully perched on a sunlit cliff overlooking a tranquil sea, her hair gently tousled by the breeze. The scene is infused with a sense of peace, evoking a dreamlike atmosphere, reminiscent of Impressionist paintings."
height = 4096
width = 4096
image = pipe(
    prompt,
    height=height,
    width=width,
    guidance_scale=3.5,
    num_inference_steps=28,
    max_sequence_length=512,
    generator=torch.manual_seed(0),
    ntk_factor=10,
    proportional_attention=True
).images[0]
image